In [1]:
import models
import utils
import torch
import torch.nn as nn
import torch.nn.functional as f
import numpy as np
import matplotlib.pyplot as plt
from torch import optim
from torch.autograd import Variable


%load_ext autoreload
%autoreload 2

torch.manual_seed(42)

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

Y = torch.tensor(np.load('../datasets/floored_exp_correl.npy'), dtype=dtype)
A_true = np.load('../datasets/students_correl.npy')
D_true = np.load('../datasets/questions_correl.npy')
# We assume we know the relevant concept of each question beforehand
concepts = np.nonzero(D_true)
num_students, num_concepts = A_true.shape
num_questions = D_true.shape[0]
guess_prob = 1/5

In [2]:
def accuracy(R_pred, R_true):
    R_pred = R_pred.data.numpy()
    R_true = R_true.data.numpy()
    R_pred_cpy = np.copy(R_pred)
    R_pred_cpy[R_pred_cpy > 0.5] = 1
    R_pred_cpy[R_pred_cpy <= 0.5] = 0
    print("Accuracy: {}".format(np.sum(R_pred_cpy == R_true) / (R_true.shape[0]*R_true.shape[1])))

In [ ]:
def rmse_min_max(A,B):
    a_norm = (A - A.min())/(A.max()-A.min())
    b_norm = (B - B.min())/(B.max()-B.min())
    return 

In [3]:
Yt = Y.transpose(0,1)

## Training Simple RNN to predict students performance

In [31]:
n_epochs = 3000
hidden_size = 128

model = models.RNN_Model(hidden_size, num_layers=4)
criterion = nn.functional.binary_cross_entropy
optimizer = optim.Adam(model.parameters(), lr=0.0002, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

In [5]:
losses = np.zeros(n_epochs) # For plotting

for epoch in range(n_epochs):

    inputs = Variable(Yt[:-1])
    targets = Variable(Yt[1:])

    outputs, hidden = model(inputs, None)

    optimizer.zero_grad()
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()

    losses[epoch] += loss.data[0]

    if epoch % 50 == 0:
        print(epoch, loss.data[0])
        accuracy(outputs,targets)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


0 tensor(0.6936)
Accuracy: 0.49754545454545457


KeyboardInterrupt: 

## Training Skill RNN to predict students performance

In [4]:
n_epochs = 3000
hidden_size = 128

concepts = np.nonzero(D_true)
model = models.RNN_Skills_Model(concepts, num_concepts, num_questions, hidden_size, num_students, num_layers=4)
criterion = nn.functional.binary_cross_entropy
optimizer = optim.Adam(model.parameters(), lr=0.0002, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

In [7]:
losses = np.zeros(n_epochs) # For plotting

for epoch in range(n_epochs):

    inputs = Variable(Yt[:-1])
    targets = Variable(Yt[1:])

    outputs, hidden, skills, D = model(inputs, None)

    optimizer.zero_grad()
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()

    losses[epoch] += loss.data[0]

    if epoch % 50 == 0:
        print(epoch, loss.data[0])
        accuracy(outputs,targets)
        print('RMSE A'.format(rmse_min_max(skills.data.numpy(), A_true)))
        print('RMSE D'.format(rmse_min_max(D.data.numpy(), D_true)))

C:\Users\Juan\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:15: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  from ipykernel import kernelapp as app
C:\Users\Juan\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:18: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


0 tensor(1.1804)
Accuracy: 0.5094444444444445
RMSE A
RMSE D


KeyboardInterrupt: 